# LELA32051 Training a Perceptron


In [ ]:
!wget https://raw.githubusercontent.com/cbannard/compling23/main/CL_Week_4_Materials/model.pth
!wget https://raw.githubusercontent.com/cbannard/compling23/main/CL_Week_4_Materials/nn_tools.py
!wget https://raw.githubusercontent.com/cbannard/compling23/main/CL_Week_4_Materials/nn_tools2.py
!wget https://raw.githubusercontent.com/cbannard/compling23/main/CL_Week_4_Materials/reviews_with_splits_lite.csv
!wget https://raw.githubusercontent.com/cbannard/compling23/main/CL_Week_4_Materials/vectorizer.json


In [ ]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from nn_tools import Vocabulary, ReviewVectorizer, ReviewDataset, ReviewClassifier
from nn_tools2 import *

import matplotlib.pyplot as plt
%matplotlib inline

### Create Dummy Data for a classifier
Imagine a set of one word reviews of products using a vocabulary  with two semantic *dimensions* and two possible labels (negative or positive)

In [ ]:
WORD1_CENTER = (3, 3)
WORD2_CENTER = (3, -2)
def get_toy_data(batch_size, w1_center=WORD1_CENTER, w2_center=WORD2_CENTER):
      x_data = []
      y_targets = np.zeros(batch_size)
      for batch_i in range(batch_size):
          if np.random.random() > 0.5:
              x_data.append(np.random.normal(loc=w1_center))
          else:
              x_data.append(np.random.normal(loc=w2_center))
              y_targets[batch_i] = 1
      return torch.tensor(x_data, dtype=torch.float32), torch.tensor(y_targets, dtype=torch.float32)

### Plot data

In [ ]:
seed = 1337

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

x_data, y_truth = get_toy_data(batch_size=1000)

x_data = x_data.data.numpy()
y_truth = y_truth.data.numpy()

left_x = []
right_x = []
left_colors = []
right_colors =  []

for x_i, y_true_i in zip(x_data, y_truth):
    color = 'black'

    if y_true_i == 0:
        left_x.append(x_i)
        left_colors.append(color)

    else:
        right_x.append(x_i)
        right_colors.append(color)

left_x = np.stack(left_x)
right_x = np.stack(right_x)

_, ax = plt.subplots(1, 1, figsize=(10,4))

ax.scatter(left_x[:, 0], left_x[:, 1], color=left_colors, marker='*', s=100)
ax.scatter(right_x[:, 0], right_x[:, 1], facecolor='white', edgecolor=right_colors, marker='o', s=100)

plt.axis('off');

### Defining our Perceptron


In [ ]:
class Perceptron(nn.Module):
    """ A Perceptron is one Linear layer """

    def __init__(self, input_dim):
        """
        Args:
            input_dim (int): size of the input features
        """
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)

    def forward(self, x_in):
        """The forward pass of the MLP

        Args:
            x_in (torch.Tensor): an input data tensor.
                x_in.shape should be (batch, input_dim)
        Returns:
            the resulting tensor. tensor.shape should be (batch, 1)
        """
        return torch.sigmoid(self.fc1(x_in))

We can then train that Perceptron to assign labels to our 1 word reviews

In [ ]:
lr = 0.01
input_dim = 2

batch_size = 1000
n_epochs = 12
n_batches = 5

seed = 1337

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

perceptron = Perceptron(input_dim=input_dim)
optimizer = optim.Adam(params=perceptron.parameters(), lr=lr)
bce_loss = nn.BCELoss()

losses = []

x_data_static, y_truth_static = get_toy_data(batch_size)
fig, ax = plt.subplots(1, 1, figsize=(10,5))
visualize_results(perceptron, x_data_static, y_truth_static, ax=ax, title='Initial Model State')
plt.axis('off')

change = 1.0
last = 10.0
epsilon = 1e-3
epoch = 0
while change > epsilon or epoch < n_epochs or last > 0.3:
#for epoch in range(n_epochs):
    for _ in range(n_batches):

        optimizer.zero_grad()
        x_data, y_target = get_toy_data(batch_size)
        y_pred = perceptron(x_data).squeeze()
        loss = bce_loss(y_pred, y_target)
        loss.backward()
        optimizer.step()


        loss_value = loss.item()
        losses.append(loss_value)

        change = abs(last - loss_value)
        last = loss_value

    fig, ax = plt.subplots(1, 1, figsize=(10,5))
    visualize_results(perceptron, x_data_static, y_truth_static, ax=ax, epoch=epoch,
                      title=f"{loss_value}; {change}")
    plt.axis('off')
    epoch += 1
    #plt.savefig('epoch{}_toylearning.png'.format(epoch))